In [ ]:
import matplotlib.pylab as plt
import numpy as np

single_point = [0,0]
# list of points on a circle around the origin
circle_points = []
for i in range(0, 380, 20):
    circle_points.append([np.cos(np.deg2rad(i))/2, np.sin(np.deg2rad(i))/2])

circle_points = np.array(circle_points)

# create circle shell around origi

# Generate 3D data
phi_values = np.linspace(0, 2 * np.pi, 10)
theta_values = np.linspace(0, np.pi, 10)
phi, theta = np.meshgrid(phi_values, theta_values)
x_3d = np.sin(theta) * np.cos(phi)
y_3d = np.sin(theta) * np.sin(phi)
z_3d = np.cos(theta)


fig, axes = plt.subplots(1,3, figsize=(15,5))
axes[0].plot(single_point[0], single_point[1], 'o', color='black', markersize=20)
axes[0].set_xticks([])
axes[0].set_yticks([])
axes[1].plot(circle_points[:,0], circle_points[:,1], 'o', color='black', markersize=20)
axes[1].plot(circle_points[:,0], circle_points[:,1], '-', color='black', markersize=10)
axes[1].set_xticks([])
axes[1].set_yticks([])
axes[2].set_xticks([])
axes[2].set_yticks([])
axes_3d = fig.add_subplot(1,3,3, projection='3d')
axes_3d.plot_surface(x_3d,y_3d,z_3d, cmap='gray')
axes_3d.view_init(30, 30)
axes_3d.axis('off')
axes_3d.set_xticks([])
axes_3d.set_yticks([])
plt.tight_layout()
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(single_point[0], single_point[1], 'o', color='black', markersize=20)
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(circle_points[:,0], circle_points[:,1], 'o', color='black', markersize=20)
plt.plot(circle_points[:,0], circle_points[:,1], '-', color='black', markersize=10)
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
axes_3d = plt.subplot(1,1,1, projection='3d')
axes_3d.plot_surface(x_3d,y_3d,z_3d, cmap='gray')
axes_3d.view_init(30, 30)
axes_3d.axis('off')
axes_3d.set_xticks([])
axes_3d.set_yticks([])
plt.show()